In [39]:
import PyPDF2

def extract_english_text(pdf_file):
    english_text = ""

    # Open the PDF file
    with open(pdf_file, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Loop through each page in the PDF
        for page_num in range(pdf_reader.getNumPages()):
            page = pdf_reader.getPage(page_num)

            # Extract text from the page
            page_text = page.extract_text()

            # Check if the extracted text contains English characters
            if any(char.isalpha() for char in page_text):
                english_text += page_text

    return english_text

if __name__ == "__main__":
    pdf_file = "../sih_telugu/english_school/School-Teachers-Training-Module.pdf"  # Replace with the path to your PDF file
    extracted_text = extract_english_text(pdf_file)

    # Print or use the extracted English text as needed
    print(extracted_text)


DeprecationError: reader.getNumPages is deprecated and was removed in PyPDF2 3.0.0. Use len(reader.pages) instead.

In [37]:
from fpdf import FPDF
from PIL import Image
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from PyPDF2 import PdfReader, PdfWriter
from googletrans import Translator

# Register the Telugu font
pdfmetrics.registerFont(TTFont('Telugu', '../sih_telugu/Gautami.ttf'))

# Function to create a PDF with justified text and double line spacing
def create_pdf(text, output_filename):
    # Create a SimpleDocTemplate with the given output file
    doc = SimpleDocTemplate(output_filename, pagesize=letter)

    # Create a list to hold the content
    story = []

    # Define a paragraph style with left alignment
    styles = getSampleStyleSheet()
    para_style = styles['Normal']
    para_style.alignment = 0  # 0 corresponds to 'left' alignment

    # Split the text into lines to fit the page width
    text_lines = text.split('\n')

    # Add each line as a Paragraph with double line spacing
    for line in text_lines:
        formatted_line = f'<font name="Telugu" size="12">{line}</font>'
        para = Paragraph(formatted_line, para_style)
        story.append(para)
        story.append(Spacer(1, 24))  # Double line spacing (adjust as needed)

    # Build the PDF document
    doc.build(story)

# Function to translate and create the PDF
def translate_and_create_pdf(input_pdf_path, output_pdf_path):
    # Initialize the PDF reader and translator
    reader = PdfReader(input_pdf_path)
    translator = Translator()

    # Create a PDF writer to write the translated pages
    writer = PdfWriter()

    # Create a PDF with a blue background and text
    pdf = FPDF()
    pdf.add_page()
    img = Image.new('RGB', (210, 297), "#afeafe")
    img.save('blue_colored.png')
    pdf.image('blue_colored.png', x=0, y=0, w=210, h=297, type='', link='')
    pdf.set_font("Arial", size=12)
    pdf.cell(ln=200, h=40, align='L', w=0, border=0, fill=False)
    pdf.output('temp_page.pdf', 'F')

    for page_number, page in enumerate(reader.pages):
        # Extract text from the page
        text = page.extract_text()

        # Translate the text
        result = translator.translate(text, src='en', dest='te')
        translated_text = result.text

        # Create a new page with the translated text and blue background
        create_pdf(translated_text, 'temp_page2.pdf')

        # Merge the blue background page with the translated text page
        page_with_background = PdfReader('temp_page.pdf').pages[0]
        page_with_text = PdfReader('temp_page2.pdf').pages[0]
        page_with_background.merge_page(page_with_text)

        # Add the merged page to the writer
        writer.add_page(page_with_background)

    # Save the final PDF with translated pages
    with open(output_pdf_path, 'wb') as output_file:
        writer.write(output_file)

    print(f"Translated PDF created: {output_pdf_path}")

if __name__ == "__main__":
    input_pdf = 'english.pdf'
    output_pdf = 'main.pdf'
    translate_and_create_pdf(input_pdf, output_pdf)


Translated PDF created: main.pdf
